In [1]:
import os
import re 
import jieba
from bs4 import BeautifulSoup

In [10]:
def runUdnSegment(dataroot, outputfile, action, pattern):
    '''Extract UDN news, and ouput char-level segement file 
    
    Need to revise pattern regular experission 
    
    Args:
        dataroot (str): the position of UDN news (recursive)
        outputfile (str): the position of output file 
        action (dict): parameter for seperate and segmentation-level 
        pattern (rgex): pattern for kicking unwanted character 
    Return: 
        None
    '''
    
    sep_method = action.get('sep_method', 'comma')
    seg_level = action.get('seg_level', 'char')
    
    if os.path.exists(outputfile): 
        print('Clean %s' %(outputfile))
        os.remove(outputfile)
    
    pattern = re.compile('TEXT')
    for dirPath, dirName, filelist in os.walk(dataroot, topdown=False):
        if pattern.search(dirPath):        
            print(dirPath)
            for file in filelist:
                inputfile = dirPath+'/'+file
    #             print(inputfile)
                with open(inputfile, 'rb') as fp:
                    data = fp.read().decode('big5-hkscs', 'ignore')
                    soup = BeautifulSoup(data)
                    
                content = contentExtract(soup)
                seqs = seperateSeq(content, sep_method)
                seqs_filter = filterSeq(seqs, pattern)
                seg_string = transformSeq(seqs_filter, seg_level)
                

                with open(outputfile, 'a', encoding='utf8') as wp:
                    wp.write(seg_string+'\n')

In [3]:
def contentExtract(soup):
    '''Extract the string content of website 
    Args:
        soup (Beatutifulsoup): website 
    Return:
        output (str): the string content of website
    '''
    output = ''
    for pTxt in soup.find_all('p'):
        res = ''
        for tag_c in pTxt.contents:
            try:
                if tag_c.get('class')==1:
                    res = res+tag_c.string
            except:
                res = res + tag_c
        res = res.strip('.\f\n\r\t\v')
        if len(res)==0:
            continue
        output = output + res+'\n'
#         print(output)
    return output

In [4]:
def seperateSeq(content, sep_method):
    '''Seperate string into sub-sentence
    Args:
        content (str): website content 
        sep_method (str): 'original' or 'comma'
    Return:
        output (list): sub-sentence
    '''
    output = []
    if sep_method == 'comma':
        pattern = re.compile('[，。！？]')
        content = ''.join(content.split('\n'))

        pre_idx=0
        for idx, ch in enumerate(content):
            if pattern.search(ch):
                tmp = content[pre_idx:idx+1]
                output.append(tmp)
                pre_idx = idx+1

    #     print(pre_idx, len(seq))
        if pre_idx<len(content):
            tmp = content[pre_idx:len(content)]
            output.append(tmp)
    elif sep_method:
        output = content.split('\n')
    
    return output 

In [16]:
def filterSeq(lst, pattern):
    '''Filter unwanted sequence based on pattern
    Args:
        lst (list): the list of website seperated content 
        pattern (rgex): the pattern we don't want 
    Return:
        output (list): list after filter
    '''
    
    output = [seq for seq in lst if not pattern.search(seq)]
    
    if not output:
        return list()
    
    if output[0].find('】'): 
        _ = output.pop(0)
    
    return output 

In [6]:
def transformSeq(seqs, seg_level):
    '''filter the line existed Unwatned pattern, and seperate the char with "space"
    Args:
        seqs (list): list from website 
        seg_level: which lm-level ('word' OR 'char)
    Return:
        output (str): string which have been seperated by 'space'
    '''
    output = list()
    if seg_level == 'word':
        for seq in seqs:
            segs = jieba.cut(seq)
            output.append(' '.join(segs))            
    elif seg_level == 'char':
        for seq in seqs:
            output.append(' '.join(seq))
    return '\n'.join(output)

In [7]:
def runSinica(inputfile, outputfile):
    with open(inputfile, 'r', encoding='utf8') as fp,\
    open(outputfile, 'w', encoding='utf8') as wp:
        for line in fp:
            data = line.strip('\n').split(' ')
            output = []
            for item in data:
                tmp = item.split('|')
                if len(tmp)==2:
                    output.append(tmp[0])

            wp.write(' '.join(output))
            wp.write('\n')
    #         print(' '.join(output))
        

In [17]:
if __name__=='__main__':
    ptn = re.compile('[A-Za-z0-9.\s]')
    par = {
        'sep_method':'comma'
        , 'seg_level':'char'}
    
    runUdnSegment(dataroot='G:/UDN/Files/', outputfile='G:/UDN/lm_data/UDN_0507Train.txt',
                 action=par, pattern=ptn)
    

#     dataroot = '/home/kiwi/Documents/udn_data/Files/'
#     runCharSegment(dataroot,outputfile,pattern)
#     runWordSegment(dataroot,'./lm_data/seg_all.txt')
#     inputfile = '/home/kiwi/udn_data/training/sinica.corpus.txt'
#     runSinica(inputfile,'./lm_data/sinica_word.txt')

Clean G:/UDN/lm_data/UDN_0507Train.txt
G:/UDN/Files/20160610\TEXT


F:\Program Files\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file F:\Program Files\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


G:/UDN/Files/20160611\TEXT
G:/UDN/Files/20160612\TEXT
G:/UDN/Files/20160613\TEXT
G:/UDN/Files/20160614\TEXT
G:/UDN/Files/20160615\TEXT
G:/UDN/Files/20160616\TEXT
G:/UDN/Files/20160617\TEXT
G:/UDN/Files/20160618\TEXT
G:/UDN/Files/20160619\TEXT
G:/UDN/Files/20160620\TEXT
G:/UDN/Files/20160621\TEXT
G:/UDN/Files/20160622\TEXT
G:/UDN/Files/20160623\TEXT
G:/UDN/Files/20160624\TEXT
G:/UDN/Files/20160625\TEXT
G:/UDN/Files/20160626\TEXT
G:/UDN/Files/20160627\TEXT


F:\Program Files\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://theinitium.com/art
icle/20160313-culture-colu
mn-liuzijie/
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


G:/UDN/Files/20160628\TEXT
G:/UDN/Files/20160629\TEXT
G:/UDN/Files/20160630\TEXT
G:/UDN/Files/20160701\TEXT


KeyboardInterrupt: 